# Herramientas para Mejora de Accesibilidad Web

Este notebook contiene herramientas para identificar y corregir problemas de accesibilidad web en archivos HTML. Utilizaremos BeautifulSoup para analizar y modificar el código HTML, enfocándonos en los problemas de accesibilidad detectados por SonarLint.

## 1. Importar Bibliotecas Necesarias

En esta sección importaremos BeautifulSoup para el análisis HTML, re para expresiones regulares, y otras bibliotecas útiles para el procesamiento de archivos.

In [ ]:
import os
import re
from bs4 import BeautifulSoup
try:
    import html5lib  # Más permisivo con HTML mal formado
    parser = 'html5lib'
except ImportError:
    parser = 'html.parser'
    print("html5lib no está instalado. Usando html.parser predeterminado.")

import codecs  # Para manejar diferentes codificaciones
import json
from typing import List, Dict, Any, Optional, Tuple
from pathlib import Path

## 2. Cargar y Analizar el Archivo HTML

Ahora cargaremos el archivo HTML de la plantilla con problemas de accesibilidad y lo analizaremos con BeautifulSoup para manipular su estructura.

In [ ]:
# Definimos la ruta al archivo HTML con problemas de accesibilidad
html_file_path = "/Users/edefrutos/__Proyectos/edf_catalogotablas/app/templates/catalogos/view_broken.html"

# Verificamos si el archivo existe
if not os.path.exists(html_file_path):
    raise FileNotFoundError(f"El archivo {html_file_path} no existe")

# Leemos el contenido del archivo
with open(html_file_path, "r") as file:
    html_content = file.read()

# Analizamos el HTML con BeautifulSoup
soup = BeautifulSoup(html_content, parser)

# Información básica
print(f"Archivo cargado: {html_file_path}")
print(f"Tamaño del archivo: {len(html_content)} bytes")
print(f"Número de elementos en el documento: {len(soup.find_all())}")
print(f"Parser utilizado: {parser}")

## 3. Identificar Problemas de Accesibilidad

En esta sección desarrollaremos funciones para identificar problemas comunes de accesibilidad como enlaces sin texto descriptivo, elementos sin atributos ARIA adecuados, y otros problemas según WCAG.

In [ ]:
def identificar_problemas_accesibilidad(soup):
    """
    Identifica problemas de accesibilidad comunes en un documento HTML.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML analizado
    
    Returns:
        Dict[str, List]: Diccionario con los problemas identificados por categoría
    """
    problemas = {
        "enlaces_no_accesibles": [],
        "elementos_no_interactivos_con_eventos": [],
        "enlaces_como_botones": [],
        "multimedia_sin_accesibilidad": [],
        "alt_texto_redundante": [],
        "roles_aria_incorrectos": [],
        "codigo_comentado": []
    }
    
    # 1. Enlaces no accesibles (sin texto o con texto genérico)
    for a in soup.find_all("a"):
        texto = a.get_text(strip=True)
        if not texto or texto in ["Haz clic aquí", "Click", "Ver", "Aquí"]:
            problemas["enlaces_no_accesibles"].append(str(a)[:100] + "...")
    
    # 2. Elementos no interactivos con eventos de ratón
    for elem in soup.find_all(["span", "div", "img"]):
        if elem.has_attr("onclick") and not (elem.has_attr("onkeypress") or elem.has_attr("onkeydown") or elem.has_attr("onkeyup")):
            problemas["elementos_no_interactivos_con_eventos"].append(str(elem)[:100] + "...")
    
    # 3. Enlaces usados como botones
    for a in soup.find_all("a"):
        href = a.get("href", "")
        onclick = a.get("onclick", "")
        if href == "#" or href == "javascript:void(0)" or "event.preventDefault()" in onclick:
            problemas["enlaces_como_botones"].append(str(a)[:100] + "...")
    
    # 4. Multimedia sin accesibilidad (videos sin subtítulos, etc.)
    for video in soup.find_all("video"):
        if not video.find("track", attrs={"kind": ["captions", "subtitles"]}):
            problemas["multimedia_sin_accesibilidad"].append(str(video)[:100] + "...")
    
    # 5. Textos alternativos redundantes
    for img in soup.find_all("img"):
        alt = img.get("alt", "")
        if alt and ("imagen" in alt.lower() or "foto" in alt.lower() or "multimedia" in alt.lower()):
            problemas["alt_texto_redundante"].append(str(img)[:100] + "...")
    
    # 6. Roles ARIA incorrectos
    for elem in soup.find_all(attrs={"role": True}):
        role = elem.get("role")
        if role == "status" and elem.name != "output":
            problemas["roles_aria_incorrectos"].append(str(elem)[:100] + "...")
    
    # 7. Código comentado
    for comentario in soup.find_all(string=lambda text: isinstance(text, str) and text.strip().startswith("<!--") and not ("DEBUG" in text or "TODO" in text)):
        problemas["codigo_comentado"].append(str(comentario)[:100] + "...")
    
    return problemas

# Ejecutamos la función para identificar problemas
problemas_encontrados = identificar_problemas_accesibilidad(soup)

# Mostramos un resumen de los problemas
for categoria, items in problemas_encontrados.items():
    print(f"{categoria}: {len(items)} problemas encontrados")

# Mostramos ejemplos de cada categoría de problema
for categoria, items in problemas_encontrados.items():
    if items:
        print(f"\nEjemplos de {categoria}:")
        for i, item in enumerate(items[:3]):  # Limitamos a 3 ejemplos
            print(f"  {i+1}. {item}")
        if len(items) > 3:
            print(f"  ... y {len(items)-3} más.")

## 4. Arreglar Enlaces con javascript:void(0)

En esta sección vamos a reemplazar los enlaces con javascript:void(0) o con href="#" por alternativas accesibles que utilizan botones o elementos interactivos apropiados con manejadores de eventos.

In [ ]:
def convertir_enlaces_a_botones(soup):
    """
    Convierte enlaces utilizados como botones a elementos <button> apropiados
    
    Args:
        soup: Objeto BeautifulSoup con el HTML analizado
    
    Returns:
        int: Número de enlaces convertidos
    """
    enlaces_convertidos = 0
    
    # Buscamos enlaces que son realmente botones
    for a in soup.find_all("a"):
        href = a.get("href", "")
        onclick = a.get("onclick", "")
        
        # Verificamos si es un enlace usado como botón
        if href == "#" or href == "javascript:void(0)" or "event.preventDefault()" in onclick:
            # Creamos un nuevo elemento button
            button = soup.new_tag("button", type="button")
            
            # Copiamos las clases del enlace
            if a.has_attr("class"):
                button["class"] = a["class"]
            
            # Copiamos el contenido
            button.string = a.get_text()
            
            # Copiamos otros atributos (excepto href)
            for attr, value in a.attrs.items():
                if attr != "href":
                    button[attr] = value
            
            # Si tiene onclick con preventDefault, lo limpiamos
            if onclick and "event.preventDefault()" in onclick:
                nuevo_onclick = onclick.replace("event.preventDefault();", "").strip()
                if nuevo_onclick:
                    button["onclick"] = nuevo_onclick
            
            # Agregamos aria-label si no tiene
            if not button.has_attr("aria-label"):
                texto = button.get_text(strip=True)
                if texto:
                    button["aria-label"] = texto
            
            # Reemplazamos el enlace por el botón
            a.replace_with(button)
            enlaces_convertidos += 1
    
    return enlaces_convertidos

# Hacemos una copia del soup para no modificar el original
soup_modificado = BeautifulSoup(str(soup), parser)

# Convertimos los enlaces a botones
enlaces_convertidos = convertir_enlaces_a_botones(soup_modificado)
print(f"Se convirtieron {enlaces_convertidos} enlaces a botones")

## 5. Asegurar Atributos ARIA Apropiados

Ahora añadiremos atributos ARIA necesarios como aria-label, aria-describedby y roles para mejorar la accesibilidad para tecnologías de asistencia.

In [ ]:
def agregar_atributos_aria(soup):
    """
    Agrega atributos ARIA apropiados a elementos que los necesitan para mejorar la accesibilidad.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML a procesar
        
    Returns:
        soup: Objeto BeautifulSoup modificado con atributos ARIA añadidos
    """
    # 1. Asegurar que todos los elementos con 'role' tengan los atributos ARIA correspondientes
    elementos_con_roles = soup.find_all(attrs={"role": True})
    for elemento in elementos_con_roles:
        role = elemento.get('role')
        
        # Botones necesitan aria-pressed si son toggle buttons
        if role == "button" and 'class' in elemento.attrs:
            clases = elemento['class']
            if isinstance(clases, list) and any('toggle' in c.lower() for c in clases):
                if not elemento.has_attr('aria-pressed'):
                    elemento['aria-pressed'] = 'false'
        
        # Tabs y tabpanels necesitan aria-labelledby y aria-controls
        if role == "tab":
            if not elemento.has_attr('aria-selected'):
                elemento['aria-selected'] = 'false'
            
            # Si tiene un ID, buscar el tabpanel correspondiente
            if elemento.has_attr('id'):
                panel_id = f"panel-{elemento['id']}"
                panel = soup.find(id=panel_id)
                if panel and not elemento.has_attr('aria-controls'):
                    elemento['aria-controls'] = panel_id
                    
        if role == "tabpanel":
            if not elemento.has_attr('aria-labelledby'):
                # Buscar el tab correspondiente
                if elemento.has_attr('id') and elemento['id'].startswith('panel-'):
                    tab_id = elemento['id'][6:]  # Eliminar 'panel-'
                    tab = soup.find(id=tab_id)
                    if tab:
                        elemento['aria-labelledby'] = tab_id
    
    # 2. Agregar aria-label a elementos que lo necesitan
    # Enlaces sin texto visible
    enlaces = soup.find_all('a')
    for enlace in enlaces:
        if not enlace.get_text(strip=True) and not enlace.has_attr('aria-label'):
            # Verificar si tiene imágenes con alt que se puedan usar como texto
            img = enlace.find('img')
            if img and img.has_attr('alt'):
                enlace['aria-label'] = img['alt']
            elif enlace.has_attr('title'):
                enlace['aria-label'] = enlace['title']
            # Si tiene un icono, tratar de determinar su propósito
            elif enlace.find(['i', 'span']) and enlace.has_attr('href'):
                enlace['aria-label'] = f"Enlace a {enlace['href']}"
    
    # 3. Formularios: asegurar que todos los inputs tengan etiquetas asociadas
    inputs = soup.find_all(['input', 'select', 'textarea'])
    for input_elem in inputs:
        # Verificar si ya tiene un id que esté asociado a un label
        tiene_label_asociado = False
        if input_elem.has_attr('id'):
            label = soup.find('label', attrs={"for": input_elem['id']})
            tiene_label_asociado = label is not None
        
        if not tiene_label_asociado:
            # Si no tiene label asociado, verificar si necesita aria-label
            if (input_elem.name == 'input' and 
                input_elem.get('type') not in ['submit', 'reset', 'button', 'hidden']):
                if not input_elem.has_attr('aria-label') and not input_elem.has_attr('aria-labelledby'):
                    # Usar placeholder o name como último recurso
                    if input_elem.has_attr('placeholder'):
                        input_elem['aria-label'] = input_elem['placeholder']
                    elif input_elem.has_attr('name'):
                        input_elem['aria-label'] = input_elem['name'].replace('_', ' ').title()
    
    # 4. Asegurar que los elementos con aria-hidden="true" no contengan elementos focusables
    elementos_ocultos = soup.find_all(attrs={"aria-hidden": "true"})
    for elem_oculto in elementos_ocultos:
        elementos_focusables = elem_oculto.find_all(['a', 'button', 'input', 'select', 'textarea'])
        for elem_focusable in elementos_focusables:
            if not elem_focusable.has_attr('tabindex'):
                elem_focusable['tabindex'] = '-1'
    
    return soup

# Ejemplo de uso:
# soup = agregar_atributos_aria(soup)

## 6. Corregir Superposición de Elementos

Ahora vamos a identificar y corregir elementos que pueden estar superpuestos o que utilizan posicionamiento absoluto sin considerar la accesibilidad.

In [ ]:
def corregir_superposicion_elementos(soup):
    """
    Identifica y corrige problemas de elementos superpuestos o con posicionamiento absoluto
    que pueden afectar la accesibilidad.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML a procesar
        
    Returns:
        soup: Objeto BeautifulSoup modificado con correcciones de superposición
    """
    # 1. Identificar elementos con posicionamiento absoluto o fijo
    elementos_problematicos = []
    
    # Buscar elementos con estilos inline que contengan position:absolute o position:fixed
    elementos_con_estilo = soup.find_all(attrs={"style": True})
    for elemento in elementos_con_estilo:
        estilo = elemento.get('style', '').lower()
        if 'position:absolute' in estilo.replace(' ', '') or 'position: absolute' in estilo or \
           'position:fixed' in estilo.replace(' ', '') or 'position: fixed' in estilo:
            elementos_problematicos.append(elemento)
    
    # 2. Buscar elementos con clases comunes que suelen tener posicionamiento absoluto
    clases_comunes_absolutas = ['modal', 'popup', 'tooltip', 'overlay', 'dropdown', 'fixed']
    
    for clase in clases_comunes_absolutas:
        elementos = soup.find_all(class_=lambda x: x and clase in x.split())
        elementos_problematicos.extend(elementos)
    
    # 3. Corregir los problemas de superposición
    for elemento in elementos_problematicos:
        # Asegurarse de que los elementos superpuestos sean accesibles por teclado
        if elemento.name in ['a', 'button', 'input', 'select', 'textarea'] and not elemento.has_attr('tabindex'):
            elemento['tabindex'] = '0'
        
        # Asegurarse de que tengan etiquetas descriptivas
        if not elemento.has_attr('aria-label') and not elemento.has_attr('aria-labelledby'):
            # Si es un modal o popup, darle una etiqueta apropiada
            if elemento.has_attr('class'):
                clases = elemento['class'] if isinstance(elemento['class'], list) else [elemento['class']]
                
                for c in clases:
                    if c.lower() in ['modal', 'popup', 'dialog']:
                        elemento['role'] = 'dialog'
                        if elemento.has_attr('id'):
                            elemento['aria-labelledby'] = f"{elemento['id']}-title"
                            # Buscar si existe algún título dentro del modal
                            titulo = elemento.find(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
                            if titulo and not titulo.has_attr('id'):
                                titulo['id'] = f"{elemento['id']}-title"
                        elif not elemento.has_attr('aria-label'):
                            elemento['aria-label'] = 'Ventana de diálogo'
                    
                    elif c.lower() == 'tooltip':
                        elemento['role'] = 'tooltip'
                    
                    elif c.lower() == 'overlay':
                        # Si es un overlay que cubre la página, asegurarse que sea semántico
                        elemento['aria-hidden'] = 'true' 
    
        # Para modales y diálogos, asegurar accesibilidad de teclado
        if elemento.has_attr('role') and elemento['role'] == 'dialog':
            # Asegurarse de que el modal sea accesible por teclado y que se pueda cerrar
            close_button = elemento.find(class_=lambda x: x and 'close' in x.split())
            if close_button:
                if not close_button.has_attr('aria-label'):
                    close_button['aria-label'] = 'Cerrar'
    
    return soup

# Ejemplo de uso:
# soup = corregir_superposicion_elementos(soup)

## 7. Añadir Texto Alternativo a Elementos Multimedia

Ahora vamos a asegurarnos de que todas las imágenes y otros elementos multimedia tengan texto alternativo apropiado.

In [ ]:
def agregar_texto_alternativo(soup):
    """
    Agrega texto alternativo a imágenes y elementos multimedia que no lo tienen.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML a procesar
        
    Returns:
        soup: Objeto BeautifulSoup modificado con texto alternativo añadido
    """
    # 1. Procesar imágenes sin atributo alt
    imagenes = soup.find_all('img')
    for img in imagenes:
        if not img.has_attr('alt'):
            # Intentar determinar un texto alternativo apropiado
            alt_text = ""
            
            # Verificar si la imagen tiene un título
            if img.has_attr('title'):
                alt_text = img['title']
            # Verificar si la imagen está dentro de un enlace y usar su texto
            elif img.parent.name == 'a' and img.parent.get_text(strip=True):
                alt_text = img.parent.get_text(strip=True)
            # Verificar si la imagen tiene un nombre de archivo descriptivo
            elif img.has_attr('src'):
                src = img['src']
                # Extraer el nombre del archivo sin la extensión
                file_name = src.split('/')[-1].split('.')[0]
                # Convertir guiones y guiones bajos a espacios
                file_name = file_name.replace('-', ' ').replace('_', ' ')
                # Convertir a formato título (primera letra de cada palabra en mayúscula)
                alt_text = ' '.join(word.capitalize() for word in file_name.split())
                
                # Si el nombre es demasiado genérico, usar algo más descriptivo
                generico = ['image', 'img', 'pic', 'picture', 'photo', 'icon']
                if alt_text.lower() in generico or len(alt_text) < 3:
                    alt_text = "Imagen en la página"
            else:
                # Si no hay forma de determinar un texto significativo
                alt_text = "Imagen en la página"
            
            # Asignar el texto alternativo
            img['alt'] = alt_text
        
        # Si la imagen es puramente decorativa, usar alt vacío
        if img.has_attr('class'):
            clases = img['class'] if isinstance(img['class'], list) else [img['class']]
            if any(c in ['decorative', 'decoration', 'background'] for c in clases):
                img['alt'] = ""
                if not img.has_attr('aria-hidden'):
                    img['aria-hidden'] = 'true'
    
    # 2. Procesar elementos de audio y video para asegurar accesibilidad
    elementos_media = soup.find_all(['audio', 'video'])
    for media in elementos_media:
        # Asegurar que tienen controles
        if not media.has_attr('controls'):
            media['controls'] = ""
        
        # Asegurar que tienen transcripciones o subtítulos
        if media.name == 'video' and not media.find('track'):
            # Verificar si hay un elemento cercano con la transcripción
            padre = media.parent
            transcripcion_cercana = False
            
            # Buscar hermanos que puedan contener transcripciones
            for sibling in list(padre.children):
                if sibling != media and sibling.name in ['div', 'p', 'section'] and sibling.get_text(strip=True):
                    if any(word in sibling.get_text(strip=True).lower() for word in ['transcript', 'transcripción', 'subtítulos']):
                        transcripcion_cercana = True
                        break
            
            # Si no hay transcripción, agregar un aviso
            if not transcripcion_cercana:
                aviso = soup.new_tag('p')
                aviso['class'] = 'transcript-warning'
                aviso.string = "Este video requiere transcripción o subtítulos para mayor accesibilidad."
                media.insert_after(aviso)
        
        # Asegurar que tienen descripciones y títulos
        if not media.has_attr('aria-label') and not media.has_attr('title'):
            if media.has_attr('src'):
                src = media['src']
                file_name = src.split('/')[-1].split('.')[0]
                file_name = file_name.replace('-', ' ').replace('_', ' ')
                media['aria-label'] = ' '.join(word.capitalize() for word in file_name.split())
            else:
                media['aria-label'] = f"{media.name.capitalize()} en la página"
    
    # 3. Procesar elementos canvas e SVG para asegurar que sean accesibles
    canvas_elements = soup.find_all('canvas')
    for canvas in canvas_elements:
        if not canvas.has_attr('aria-label') and not canvas.has_attr('role'):
            canvas['role'] = 'img'
            if not canvas.has_attr('aria-label'):
                canvas['aria-label'] = "Contenido gráfico"
    
    svg_elements = soup.find_all('svg')
    for svg in svg_elements:
        if not svg.has_attr('role'):
            svg['role'] = 'img'
        if not svg.has_attr('aria-label') and not svg.find('title') and not svg.find('desc'):
            # Agregar un título al SVG
            title_tag = soup.new_tag('title')
            title_tag.string = "Gráfico SVG"
            svg.insert(0, title_tag)
    
    return soup

# Ejemplo de uso:
# soup = agregar_texto_alternativo(soup)

## 8. Mejorar Estructura Semántica

Ahora vamos a mejorar la estructura semántica del documento, reemplazando elementos no semánticos (como divs) con elementos semánticos apropiados (como main, section, article, etc.).

In [ ]:
def mejorar_estructura_semantica(soup):
    """
    Mejora la estructura semántica del documento reemplazando elementos no semánticos
    con elementos semánticos más apropiados.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML a procesar
        
    Returns:
        soup: Objeto BeautifulSoup modificado con mejor estructura semántica
    """
    # 1. Identificar posible contenido principal si no existe
    if not soup.find('main'):
        # Buscar candidatos para el contenido principal
        candidatos = []
        
        # Buscar divs que puedan contener el contenido principal
        for div in soup.find_all('div'):
            # Contenedores comunes de contenido principal
            if div.has_attr('id') and div['id'].lower() in ['content', 'main', 'main-content', 'contenido-principal']:
                candidatos.append((div, 10))
            elif div.has_attr('class'):
                clases = div['class'] if isinstance(div['class'], list) else [div['class']]
                if any(c.lower() in ['content', 'main', 'main-content', 'contenido-principal'] for c in clases):
                    candidatos.append((div, 8))
            
            # Divs que contienen la mayor parte del contenido
            if len(div.find_all(['p', 'h1', 'h2', 'h3', 'section', 'article'])) > 5:
                candidatos.append((div, len(div.find_all(['p', 'h1', 'h2', 'h3', 'section', 'article']))))
        
        # Seleccionar el mejor candidato
        if candidatos:
            candidatos.sort(key=lambda x: x[1], reverse=True)
            mejor_candidato = candidatos[0][0]
            
            # Crear un nuevo elemento main
            main = soup.new_tag('main')
            for attr, value in mejor_candidato.attrs.items():
                main[attr] = value
            
            # Mover todos los hijos al nuevo elemento main
            for child in list(mejor_candidato.children):
                main.append(child.extract())
                
            # Reemplazar el div con el nuevo main
            mejor_candidato.replace_with(main)
    
    # 2. Identificar y crear una estructura de header si no existe
    if not soup.find('header'):
        # Buscar candidatos para el header
        candidatos_header = []
        
        # Divs al inicio del cuerpo que pueden ser headers
        body = soup.find('body')
        if body:
            # Revisar los primeros hijos del body
            for i, child in enumerate(list(body.children)[:3]):
                if child.name == 'div':
                    if child.has_attr('id') and child['id'].lower() in ['header', 'cabecera', 'top']:
                        candidatos_header.append((child, 10))
                    elif child.has_attr('class'):
                        clases = child['class'] if isinstance(child['class'], list) else [child['class']]
                        if any(c.lower() in ['header', 'cabecera', 'top', 'navbar'] for c in clases):
                            candidatos_header.append((child, 8))
                    # Headers suelen tener logos, navegación y poca cantidad de texto
                    if child.find(['img', 'nav', 'a']) and len(child.get_text(strip=True)) < 500:
                        candidatos_header.append((child, 6 - i))  # Priorizar los primeros elementos
        
        # Seleccionar el mejor candidato para header
        if candidatos_header:
            candidatos_header.sort(key=lambda x: x[1], reverse=True)
            mejor_header = candidatos_header[0][0]
            
            # Crear nuevo elemento header
            header = soup.new_tag('header')
            for attr, value in mejor_header.attrs.items():
                header[attr] = value
            
            # Mover hijos al nuevo header
            for child in list(mejor_header.children):
                header.append(child.extract())
                
            # Reemplazar el div con el nuevo header
            mejor_header.replace_with(header)
    
    # 3. Identificar y crear una estructura de footer si no existe
    if not soup.find('footer'):
        # Buscar candidatos para el footer
        candidatos_footer = []
        
        # Divs al final del cuerpo que pueden ser footers
        body = soup.find('body')
        if body:
            # Revisar los últimos hijos del body
            for i, child in enumerate(reversed(list(body.children)[:3])):
                if child.name == 'div':
                    if child.has_attr('id') and child['id'].lower() in ['footer', 'pie', 'bottom']:
                        candidatos_footer.append((child, 10))
                    elif child.has_attr('class'):
                        clases = child['class'] if isinstance(child['class'], list) else [child['class']]
                        if any(c.lower() in ['footer', 'pie', 'bottom', 'foot'] for c in clases):
                            candidatos_footer.append((child, 8))
                    # Footers suelen tener enlaces, copyright y poca cantidad de texto
                    if child.find(['a']) and 'copyright' in child.get_text(strip=True).lower():
                        candidatos_footer.append((child, 6 - i))  # Priorizar los últimos elementos
        
        # Seleccionar el mejor candidato para footer
        if candidatos_footer:
            candidatos_footer.sort(key=lambda x: x[1], reverse=True)
            mejor_footer = candidatos_footer[0][0]
            
            # Crear nuevo elemento footer
            footer = soup.new_tag('footer')
            for attr, value in mejor_footer.attrs.items():
                footer[attr] = value
            
            # Mover hijos al nuevo footer
            for child in list(mejor_footer.children):
                footer.append(child.extract())
                
            # Reemplazar el div con el nuevo footer
            mejor_footer.replace_with(footer)
    
    # 4. Identificar y mejorar la estructura de navegación
    nav_elements = soup.find_all(['div', 'ul'], class_=lambda x: x and any(c.lower() in ['nav', 'menu', 'navigation'] for c in x.split()))
    for nav_element in nav_elements:
        if nav_element.name != 'nav' and not nav_element.find_parent('nav'):
            # Crear nuevo elemento nav
            nav = soup.new_tag('nav')
            for attr, value in nav_element.attrs.items():
                nav[attr] = value
            
            # Mover hijos al nuevo nav
            for child in list(nav_element.children):
                nav.append(child.extract())
                
            # Asegurarse que la navegación tiene un propósito claro
            if not nav.has_attr('aria-label'):
                nav['aria-label'] = 'Navegación principal' if 'main' in str(nav.get('class', '')).lower() else 'Navegación'
                
            # Reemplazar el elemento original con el nuevo nav
            nav_element.replace_with(nav)
    
    # 5. Convertir divs con clases/ids específicos a elementos semánticos
    for div in soup.find_all('div'):
        if div.has_attr('id'):
            id_val = div['id'].lower()
            if any(term in id_val for term in ['article', 'articulo', 'post']):
                replace_with_semantic_element(div, 'article', soup)
            elif any(term in id_val for term in ['section', 'seccion']):
                replace_with_semantic_element(div, 'section', soup)
            elif any(term in id_val for term in ['aside', 'sidebar', 'complementary']):
                replace_with_semantic_element(div, 'aside', soup)
        
        if div.has_attr('class'):
            classes = div['class'] if isinstance(div['class'], list) else [div['class']]
            class_str = ' '.join(classes).lower()
            
            if any(term in class_str for term in ['article', 'articulo', 'post', 'entry']):
                replace_with_semantic_element(div, 'article', soup)
            elif any(term in class_str for term in ['section', 'seccion']):
                replace_with_semantic_element(div, 'section', soup)
            elif any(term in class_str for term in ['aside', 'sidebar', 'complementary']):
                replace_with_semantic_element(div, 'aside', soup)
    
    # 6. Asegurar que hay exactamente un h1 en la página
    h1_elements = soup.find_all('h1')
    if len(h1_elements) == 0:
        # Buscar el mejor candidato para ser h1
        h2_elements = soup.find_all('h2')
        if h2_elements:
            # Convertir el primer h2 en h1
            h1 = soup.new_tag('h1')
            for attr, value in h2_elements[0].attrs.items():
                h1[attr] = value
                
            # Mover contenido
            for child in list(h2_elements[0].children):
                h1.append(child.extract())
                
            h2_elements[0].replace_with(h1)
    elif len(h1_elements) > 1:
        # Convertir h1 adicionales a h2 (dejar solo el primero como h1)
        for h1 in h1_elements[1:]:
            h2 = soup.new_tag('h2')
            for attr, value in h1.attrs.items():
                h2[attr] = value
                
            # Mover contenido
            for child in list(h1.children):
                h2.append(child.extract())
                
            h1.replace_with(h2)
    
    return soup

def replace_with_semantic_element(div, new_element_name, soup):
    """
    Reemplaza un div con un elemento semántico nuevo.
    
    Args:
        div: Elemento div a reemplazar
        new_element_name: Nombre del nuevo elemento semántico ('article', 'section', etc.)
        soup: Objeto BeautifulSoup para crear el nuevo elemento
    """
    # Crear nuevo elemento semántico
    new_element = soup.new_tag(new_element_name)
    
    # Copiar atributos
    for attr, value in div.attrs.items():
        new_element[attr] = value
    
    # Mover hijos
    for child in list(div.children):
        new_element.append(child.extract())
    
    # Reemplazar div con nuevo elemento
    div.replace_with(new_element)

# Ejemplo de uso:
# soup = mejorar_estructura_semantica(soup)

## 9. Verificar y Corregir Contraste de Color

Ahora vamos a identificar problemas potenciales de contraste de color en el documento, aunque esto es más difícil de automatizar completamente y podría requerir revisión manual.

In [ ]:
def identificar_problemas_contraste(soup):
    """
    Identifica potenciales problemas de contraste de color en el documento y
    añade advertencias para revisión manual.
    
    Args:
        soup: Objeto BeautifulSoup con el HTML a procesar
        
    Returns:
        soup: Objeto BeautifulSoup con advertencias de contraste añadidas
        problemas_contraste: Lista de elementos con posibles problemas de contraste
    """
    import re
    
    problemas_contraste = []
    
    # Expresiones regulares para identificar colores en diferentes formatos
    hex_color_pattern = re.compile(r'#(?:[0-9a-fA-F]{3}|[0-9a-fA-F]{6})\b')
    rgb_color_pattern = re.compile(r'rgb\(\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*\)')
    rgba_color_pattern = re.compile(r'rgba\(\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*,\s*([01]?\.?\d*)\s*\)')
    
    # 1. Buscar elementos con estilos inline que contengan colores
    elementos_con_estilo = soup.find_all(attrs={"style": True})
    for elemento in elementos_con_estilo:
        estilo = elemento.get('style', '').lower()
        
        # Verificar si hay definiciones de color
        if 'color:' in estilo or 'background-color:' in estilo or 'background:' in estilo:
            # Extraer colores
            colores_hex = hex_color_pattern.findall(estilo)
            colores_rgb = rgb_color_pattern.findall(estilo)
            colores_rgba = rgba_color_pattern.findall(estilo)
            
            if colores_hex or colores_rgb or colores_rgba:
                # Elementos con definiciones de color específicas
                problemas_contraste.append({
                    'elemento': elemento.name,
                    'estilo': estilo,
                    'texto': elemento.get_text(strip=True)[:50] + ('...' if len(elemento.get_text(strip=True)) > 50 else '')
                })
                
                # Añadir una clase para facilitar la identificación
                if elemento.has_attr('class'):
                    if isinstance(elemento['class'], list):
                        elemento['class'].append('contraste-revisar')
                    else:
                        elemento['class'] = [elemento['class'], 'contraste-revisar']
                else:
                    elemento['class'] = 'contraste-revisar'
    
    # 2. Buscar elementos con clases que suelen tener problemas de contraste
    clases_problematicas = [
        'text-muted', 'text-light', 'text-white', 'text-secondary',
        'bg-light', 'bg-white', 'bg-transparent', 'btn-light', 'btn-outline',
        'light', 'faded', 'subtle'
    ]
    
    for clase in clases_problematicas:
        elementos = soup.find_all(class_=lambda x: x and clase in x.split())
        for elemento in elementos:
            problemas_contraste.append({
                'elemento': elemento.name,
                'clase': elemento.get('class', ''),
                'texto': elemento.get_text(strip=True)[:50] + ('...' if len(elemento.get_text(strip=True)) > 50 else '')
            })
            
            # Añadir una clase para facilitar la identificación
            if elemento.has_attr('class'):
                if isinstance(elemento['class'], list):
                    elemento['class'].append('contraste-revisar')
                else:
                    elemento['class'] = [elemento['class'], 'contraste-revisar']
            else:
                elemento['class'] = 'contraste-revisar'
    
    # 3. Añadir un comentario en el HTML para advertir sobre la revisión manual
    if problemas_contraste:
        comentario = soup.new_comment(
            " ADVERTENCIA: Se identificaron posibles problemas de contraste de color. " +
            f"Se han marcado {len(problemas_contraste)} elementos con la clase 'contraste-revisar' " +
            "para revisión manual. Se recomienda verificar que todos los elementos tengan " +
            "un contraste adecuado según las guías WCAG. "
        )
        body = soup.find('body')
        if body:
            body.insert(0, comentario)
    
    return soup, problemas_contraste

# Ejemplo de uso:
# soup, problemas_contraste = identificar_problemas_contraste(soup)
# print(f"Se identificaron {len(problemas_contraste)} posibles problemas de contraste.")
# for problema in problemas_contraste[:5]:  # Mostrar solo los primeros 5
#     print(problema)

## 10. Guardar HTML Corregido y Verificación Final

Por último, vamos a aplicar todas las correcciones y guardar el HTML accesible.

In [ ]:
def aplicar_todas_correcciones(html_path, output_path=None):
    """
    Aplica todas las correcciones de accesibilidad al archivo HTML y guarda el resultado.
    
    Args:
        html_path: Ruta al archivo HTML que se va a corregir
        output_path: Ruta donde guardar el archivo corregido. Si es None, se añade '_accesible' al nombre original
        
    Returns:
        soup: Objeto BeautifulSoup con todas las correcciones aplicadas
        informe: Diccionario con información sobre las correcciones realizadas
    """
    import os
    import re
    import codecs
    import json
    from bs4 import BeautifulSoup
    
    # Determinar el path de salida si no se proporciona
    if output_path is None:
        nombre_base, extension = os.path.splitext(html_path)
        output_path = f"{nombre_base}_accesible{extension}"
    
    # Informe de correcciones
    informe = {
        "archivo_original": html_path,
        "archivo_corregido": output_path,
        "problemas_identificados": {},
        "elementos_modificados": {}
    }
    
    try:
        # Cargar el HTML
        with codecs.open(html_path, "r", encoding='utf-8', errors='replace') as f:
            html_content = f.read()
        
        # Intentar con diferentes parsers
        try:
            # Primero intentar con html5lib (mejor manejo de HTML malformado)
            soup = BeautifulSoup(html_content, 'html5lib')
            parser_usado = 'html5lib'
        except:
            # Si falla, usar el parser estándar
            soup = BeautifulSoup(html_content, 'html.parser')
            parser_usado = 'html.parser'
            
        informe["parser_utilizado"] = parser_usado
        
        # 1. Identificar problemas de accesibilidad
        problemas = identificar_problemas_accesibilidad(soup)
        informe["problemas_identificados"] = problemas
        
        # 2. Convertir enlaces a botones
        antes_enlaces = len(soup.find_all('a', href=lambda x: x == "#" or (x and "javascript:void(0)" in x)))
        soup = convertir_enlaces_a_botones(soup)
        despues_enlaces = len(soup.find_all('a', href=lambda x: x == "#" or (x and "javascript:void(0)" in x)))
        informe["elementos_modificados"]["enlaces_convertidos_a_botones"] = antes_enlaces - despues_enlaces
        
        # 3. Agregar atributos ARIA
        antes_aria = len(soup.find_all(attrs=lambda attrs: any(attr.startswith('aria-') for attr in attrs.keys() if attrs)))
        soup = agregar_atributos_aria(soup)
        despues_aria = len(soup.find_all(attrs=lambda attrs: any(attr.startswith('aria-') for attr in attrs.keys() if attrs)))
        informe["elementos_modificados"]["elementos_con_aria_nuevos"] = despues_aria - antes_aria
        
        # 4. Corregir superposición de elementos
        antes_superposicion = len(soup.find_all(attrs={"style": True}))
        soup = corregir_superposicion_elementos(soup)
        informe["elementos_modificados"]["elementos_superpuestos_corregidos"] = "Correcciones aplicadas"
        
        # 5. Agregar texto alternativo
        antes_alt = len(soup.find_all('img', alt=True))
        soup = agregar_texto_alternativo(soup)
        despues_alt = len(soup.find_all('img', alt=True))
        informe["elementos_modificados"]["elementos_con_alt_nuevos"] = despues_alt - antes_alt
        
        # 6. Mejorar estructura semántica
        antes_semantica = {
            'main': len(soup.find_all('main')),
            'header': len(soup.find_all('header')),
            'footer': len(soup.find_all('footer')),
            'nav': len(soup.find_all('nav')),
            'article': len(soup.find_all('article')),
            'section': len(soup.find_all('section')),
            'aside': len(soup.find_all('aside'))
        }
        
        soup = mejorar_estructura_semantica(soup)
        
        despues_semantica = {
            'main': len(soup.find_all('main')),
            'header': len(soup.find_all('header')),
            'footer': len(soup.find_all('footer')),
            'nav': len(soup.find_all('nav')),
            'article': len(soup.find_all('article')),
            'section': len(soup.find_all('section')),
            'aside': len(soup.find_all('aside'))
        }
        
        informe["elementos_modificados"]["elementos_semanticos_nuevos"] = {
            tag: despues_semantica[tag] - antes_semantica[tag] for tag in antes_semantica
        }
        
        # 7. Identificar problemas de contraste
        soup, problemas_contraste = identificar_problemas_contraste(soup)
        informe["problemas_identificados"]["posibles_problemas_contraste"] = len(problemas_contraste)
        
        # Añadir un comentario con información sobre las correcciones
        correcciones_realizadas = (
            f"Correcciones realizadas: "
            f"{informe['elementos_modificados'].get('enlaces_convertidos_a_botones', 0)} enlaces convertidos a botones, "
            f"{informe['elementos_modificados'].get('elementos_con_aria_nuevos', 0)} elementos con atributos ARIA añadidos, "
            f"{informe['elementos_modificados'].get('elementos_con_alt_nuevos', 0)} imágenes con alt añadido, "
            f"y {sum(informe['elementos_modificados']['elementos_semanticos_nuevos'].values())} elementos semánticos nuevos."
        )
        
        comentario = soup.new_comment(f" Correcciones de Accesibilidad - {correcciones_realizadas} ")
        if soup.html:
            soup.html.insert(0, comentario)
        
        # Guardar el HTML corregido
        with codecs.open(output_path, "w", encoding='utf-8') as f:
            # Asegurarse de que tenga el doctype correcto
            output = "<!DOCTYPE html>\n" + str(soup)
            f.write(output)
        
        informe["estado"] = "Completado"
        
        # También guardar el informe como JSON para referencia
        informe_path = f"{os.path.splitext(output_path)[0]}_informe.json"
        with codecs.open(informe_path, "w", encoding='utf-8') as f:
            json.dump(informe, f, indent=2, ensure_ascii=False)
        
        return soup, informe
    
    except Exception as e:
        informe["estado"] = "Error"
        informe["error"] = str(e)
        return None, informe

# Ejemplo de uso:
# html_path = 'path/to/view_broken.html'
# soup, informe = aplicar_todas_correcciones(html_path)
# print(json.dumps(informe, indent=2, ensure_ascii=False))

In [ ]:
# Ejemplo de uso completo
import os
import json

# Ruta al archivo HTML a corregir - ajusta esta ruta según tu estructura de archivos
ruta_html = "view_broken.html"

# Comprobar si el archivo existe
if os.path.exists(ruta_html):
    print(f"Procesando archivo: {ruta_html}")
    soup, informe = aplicar_todas_correcciones(ruta_html)
    if informe["estado"] == "Completado":
        print(f"✅ Correcciones completadas con éxito!")
        print(f"Archivo HTML corregido guardado en: {informe['archivo_corregido']}")
        print(f"Informe de correcciones guardado en: {os.path.splitext(informe['archivo_corregido'])[0]}_informe.json")
        print("\nResumen de correcciones:")
        print(f"- {informe['elementos_modificados'].get('enlaces_convertidos_a_botones', 0)} enlaces convertidos a botones")
        print(f"- {informe['elementos_modificados'].get('elementos_con_aria_nuevos', 0)} elementos con atributos ARIA añadidos")
        print(f"- {informe['elementos_modificados'].get('elementos_con_alt_nuevos', 0)} imágenes con texto alternativo")
        
        elementos_semanticos = sum(informe['elementos_modificados']['elementos_semanticos_nuevos'].values())
        print(f"- {elementos_semanticos} elementos semánticos nuevos")
        
        if informe['problemas_identificados'].get('posibles_problemas_contraste', 0) > 0:
            print(f"⚠️ Se identificaron {informe['problemas_identificados']['posibles_problemas_contraste']} posibles problemas de contraste que requieren revisión manual")
    else:
        print(f"❌ Error al procesar el archivo: {informe.get('error', 'Error desconocido')}")
else:
    print(f"❌ El archivo {ruta_html} no existe. Asegúrate de proporcionar la ruta correcta al archivo HTML.")
    
    # Mostrar instrucciones si el archivo no existe
    print("\nPara usar este notebook correctamente:")
    print("1. Coloca el archivo HTML con problemas de accesibilidad en la misma carpeta que este notebook")
    print("2. Modifica la variable 'ruta_html' para que apunte al nombre de tu archivo")
    print("3. Ejecuta todas las celdas del notebook en orden")

## Conclusión

Este notebook proporciona un conjunto de herramientas para corregir automáticamente diversos problemas de accesibilidad en archivos HTML según las guías WCAG. Las correcciones incluyen:

1. Identificación sistemática de problemas de accesibilidad
2. Conversión de enlaces no accesibles a botones apropiados
3. Adición de atributos ARIA necesarios
4. Corrección de elementos superpuestos
5. Adición de texto alternativo a elementos multimedia
6. Mejora de la estructura semántica del documento
7. Identificación de posibles problemas de contraste de color

Para usar este notebook:
1. Coloca el archivo HTML que deseas corregir en la misma carpeta que este notebook
2. Ajusta la variable `ruta_html` para que apunte a tu archivo
3. Ejecuta todas las celdas

Las correcciones automáticas mejorarán significativamente la accesibilidad, pero se recomienda una revisión manual adicional, especialmente para los problemas de contraste de color y el contenido dinámico generado por JavaScript.

# Solución de Problemas de Accesibilidad Web en Plantillas HTML

Este cuaderno muestra cómo corregir problemas de accesibilidad detectados por SonarLint en archivos de plantillas HTML, específicamente en el archivo `view_broken.html` de la aplicación de catálogo de tablas.

## 1. Configuración del Entorno y Librerías

Para analizar y modificar código HTML, utilizaremos herramientas como BeautifulSoup y las bibliotecas estándar de Python.

In [ ]:
# Importar librerías necesarias
import os
import re
from bs4 import BeautifulSoup
import html5lib # pyright: ignore[reportMissingModuleSource]
from pathlib import Path

# Establecer la ruta del archivo HTML a analizar
html_path = "/Users/edefrutos/__Proyectos/edf_catalogotablas/app/templates/catalogos/view_broken.html"

# Función para leer el archivo HTML
def read_html_file(filepath):
    """Lee un archivo HTML y devuelve su contenido como texto"""
    with open(filepath, encoding='utf-8') as file:
        return file.read()

# Función para convertir HTML a un objeto BeautifulSoup
def html_to_soup(html_content):
    """Convierte el contenido HTML a un objeto BeautifulSoup para su análisis"""
    return BeautifulSoup(html_content, 'html5lib')

# Leer el archivo HTML
html_content = read_html_file(html_path)
print(f"Archivo cargado: {html_path}")
print(f"Tamaño: {len(html_content)} caracteres")

## 2. Análisis de Errores de Accesibilidad Comunes

SonarLint ha detectado varios problemas de accesibilidad en nuestro archivo HTML. Vamos a categorizarlos y entender su impacto:

In [ ]:
# Definir los tipos de problemas de accesibilidad detectados
problemas_accesibilidad = {
    "Web:S6848": "Elementos no nativos interactivos sin roles adecuados",
    "Web:MouseEventWithoutKeyboardEquivalentCheck": "Eventos de ratón sin equivalentes de teclado",
    "Web:S6847": "Elementos no interactivos con eventos asignados",
    "Web:S4084": "Videos sin subtítulos o descripciones",
    "Web:S6844": "Etiquetas <a> usadas como botones",
    "Web:AvoidCommentedOutCodeCheck": "Código comentado que debe eliminarse",
    "Web:S6851": "Texto alternativo redundante en imágenes",
    "Web:S6819": "Uso incorrecto de roles ARIA",
    "Web:S1135": "Comentarios TODO pendientes"
}

# Mostrar los problemas de accesibilidad y su impacto
print("Problemas de accesibilidad detectados y su impacto en usuarios:")
print("=" * 80)

for codigo, descripcion in problemas_accesibilidad.items():
    print(f"{codigo}: {descripcion}")
    
    # Explicar el impacto según el tipo de problema
    if "S6848" in codigo or "S6847" in codigo:
        print("   Impacto: Usuarios con tecnologías de asistencia no podrán interactuar con estos elementos.")
    elif "MouseEventWithoutKeyboardEquivalentCheck" in codigo:
        print("   Impacto: Usuarios que navegan solo con teclado no pueden activar estas funcionalidades.")
    elif "S4084" in codigo:
        print("   Impacto: Personas con discapacidad auditiva no pueden acceder al contenido de los videos.")
    elif "S6844" in codigo:
        print("   Impacto: Confunde a las tecnologías de asistencia y a los usuarios sobre la función del elemento.")
    elif "AvoidCommentedOutCodeCheck" in codigo:
        print("   Impacto: Reduce la mantenibilidad del código y puede causar confusión.")
    elif "S6851" in codigo:
        print("   Impacto: Lectores de pantalla pueden repetir información innecesaria.")
    elif "S6819" in codigo:
        print("   Impacto: Incompatibilidad con algunos lectores de pantalla y asistentes de voz.")
    elif "S1135" in codigo:
        print("   Impacto: Tareas incompletas que pueden afectar la funcionalidad final.")
    
    print()

## 3. Corrección de Elementos Interactivos No Nativos

Uno de los problemas más comunes es el uso de elementos HTML no interactivos (`<span>`, `<div>`, `<img>`) con eventos de clic. Veamos cómo corregir esto:

In [ ]:
# Identificar y mostrar ejemplos de elementos no nativos interactivos
def find_non_native_interactive_elements(html_content):
    """Encuentra elementos no interactivos que tienen eventos onClick"""
    soup = BeautifulSoup(html_content, 'html5lib')
    
    # Buscar elementos no interactivos con eventos onclick
    non_interactive_with_events = []
    
    for tag in ['span', 'div', 'img']:
        elements = soup.find_all(tag, onclick=True)
        for element in elements:
            non_interactive_with_events.append({
                'tag': tag,
                'html': str(element)[:100] + '...' if len(str(element)) > 100 else str(element),
                'onclick': element.get('onclick')
            })
    
    return non_interactive_with_events

# Encontrar elementos problemáticos
problematic_elements = find_non_native_interactive_elements(html_content)

# Mostrar los elementos encontrados
print(f"Se encontraron {len(problematic_elements)} elementos no nativos con eventos de clic:")
for i, elem in enumerate(problematic_elements[:3], 1):  # Mostrar solo los 3 primeros ejemplos
    print(f"\nEjemplo {i}:")
    print(f"Etiqueta: {elem['tag']}")
    print(f"HTML: {elem['html']}")
    print(f"Evento onClick: {elem['onclick']}")
print("..." if len(problematic_elements) > 3 else "")

In [ ]:
# Función para corregir los elementos <span> utilizados para ordenar tablas
def fix_span_sort_icons(html_content):
    """
    Reemplaza los elementos <span> para ordenar tablas por elementos <button> con roles apropiados
    y equivalentes de teclado
    """
    # Patrón para identificar los spans de ordenación
    pattern = r'<span class="sort-icon" data-column="(\d+)" data-type="(.*?)" onclick="sortTable\(this\.dataset\.column, this\.dataset\.type\)">\s*<i class="bi bi-chevron-expand" id="sort-icon-\d+"></i>\s*</span>'
    
    # Reemplazo con elementos <button> accesibles
    replacement = r'<button type="button" class="sort-icon-btn btn-unstyled" data-column="\1" data-type="\2" onclick="sortTable(this.dataset.column, this.dataset.type)" onkeydown="if(event.key === \'Enter\') sortTable(this.dataset.column, this.dataset.type)" aria-label="Ordenar por esta columna">\n                    <i class="bi bi-chevron-expand" id="sort-icon-\1"></i>\n                </button>'
    
    # Realizar el reemplazo en el contenido HTML
    fixed_html = re.sub(pattern, replacement, html_content)
    
    return fixed_html

# Ejemplo de corrección para uno de los elementos span
original_span = """<span class="sort-icon" data-column="0" data-type="number" onclick="sortTable(this.dataset.column, this.dataset.type)">
            <i class="bi bi-chevron-expand" id="sort-icon-0"></i>
        </span>"""

# Aplicar la corrección al ejemplo
fixed_span = fix_span_sort_icons(original_span)

# Mostrar la comparación
print("ELEMENTO ORIGINAL:")
print(original_span)
print("\nELEMENTO CORREGIDO:")
print(fixed_span)

## 4. Implementación de Equivalentes de Teclado

Muchos elementos tienen eventos de ratón (`onClick`) sin equivalentes de teclado. Vamos a añadir estos eventos para mejorar la accesibilidad:

In [ ]:
# Función para añadir eventos de teclado a elementos con onClick
def add_keyboard_events(html_content):
    """
    Añade eventos de teclado (onKeyDown) a elementos que ya tienen onClick
    pero carecen de equivalentes de teclado
    """
    soup = BeautifulSoup(html_content, 'html5lib')
    
    # Buscar elementos con onclick pero sin eventos de teclado
    elements_to_fix = []
    tags_to_check = ['img', 'div']
    
    for tag in tags_to_check:
        for element in soup.find_all(tag):
            if element.has_attr('onclick') and not (element.has_attr('onkeydown') or element.has_attr('onkeypress') or element.has_attr('onkeyup')):
                elements_to_fix.append(element)
    
    # Añadir eventos de teclado a los elementos encontrados
    for element in elements_to_fix:
        onclick_value = element['onclick']
        element['onkeydown'] = f"if(event.key === 'Enter') {{ {onclick_value} }}"
        # Añadir tabindex si no lo tiene
        if not element.has_attr('tabindex'):
            element['tabindex'] = "0"
    
    return str(soup)

# Ejemplo de elemento con click sin evento de teclado
img_example = """<img src="https://example.com/image.jpg"
     alt="Multimedia"
     class="img-thumbnail multimedia-thumbnail"
     onclick="showMultimediaModal('https://example.com/image.jpg', 'image.jpg')">"""

# Aplicar la corrección
fixed_img = add_keyboard_events(img_example)

# Mostrar la comparación
print("ELEMENTO ORIGINAL:")
print(img_example)
print("\nELEMENTO CON EVENTOS DE TECLADO:")
print(fixed_img)

## 5. Mejora de Elementos Multimedia

SonarLint detectó problemas con elementos de vídeo que carecen de subtítulos y descripciones. Vamos a corregir esto:

In [ ]:
# Función para mejorar la accesibilidad de elementos de vídeo
def enhance_video_accessibility(html_content):
    """
    Mejora los elementos de vídeo añadiendo:
    - Subtítulos (.vtt)
    - Descripciones de audio
    - Controles accesibles
    - Atributos ARIA adecuados
    """
    soup = BeautifulSoup(html_content, 'html5lib')
    
    # Buscar todos los elementos de vídeo
    videos = soup.find_all('video')
    
    for video in videos:
        # Añadir track para subtítulos (asumiendo que existen)
        video_src = None
        for source in video.find_all('source'):
            video_src = source.get('src')
            if video_src and video_src.endswith('.mp4'):
                # Generar nombre de archivo para subtítulos basado en el vídeo
                captions_path = video_src.replace('.mp4', '_captions.vtt')
                descriptions_path = video_src.replace('.mp4', '_descriptions.vtt')
                
                # Añadir track para subtítulos
                captions_track = soup.new_tag('track')
                captions_track['src'] = captions_path
                captions_track['kind'] = 'subtitles'
                captions_track['srclang'] = 'es'
                captions_track['label'] = 'Español'
                captions_track['default'] = True
                video.append(captions_track)
                
                # Añadir track para descripciones
                desc_track = soup.new_tag('track')
                desc_track['src'] = descriptions_path
                desc_track['kind'] = 'descriptions'
                desc_track['srclang'] = 'es'
                desc_track['label'] = 'Descripciones en Español'
                video.append(desc_track)
        
        # Asegurar que tiene controles
        video['controls'] = True
        
        # Añadir atributos ARIA
        video['aria-label'] = 'Video con contenido multimedia'
    
    return str(soup)

# Ejemplo de elemento de vídeo
video_example = """<video class="multimedia-thumbnail" style="pointer-events: none;">
    <source src="https://example.com/video.mp4" type="video/mp4">
    Tu navegador no soporta el elemento video.
</video>"""

# Aplicar la corrección
fixed_video = enhance_video_accessibility(video_example)

# Mostrar la comparación
print("ELEMENTO DE VÍDEO ORIGINAL:")
print(video_example)
print("\nELEMENTO DE VÍDEO MEJORADO:")
print(fixed_video)

# Nota sobre creación de archivos de subtítulos
print("\nNOTA IMPORTANTE: ")
print("Esta solución asume que existen o se crearán archivos de subtítulos (.vtt) para cada vídeo.")
print("Es necesario generar estos archivos para una verdadera accesibilidad multimedia.")

## 6. Corrección de Etiquetas de Anclaje (a) Usadas como Botones

SonarLint detectó muchos casos donde las etiquetas `<a>` se usan como botones. Esto no es semánticamente correcto y causa problemas de accesibilidad:

In [ ]:
# Función para identificar enlaces utilizados como botones
def find_links_used_as_buttons(html_content):
    """Encuentra etiquetas <a> usadas como botones (con href="#" o javascript:void(0))"""
    soup = BeautifulSoup(html_content, 'html5lib')
    links_as_buttons = []
    
    for link in soup.find_all('a'):
        href = link.get('href', '')
        onclick = link.get('onclick', '')
        
        if href == '#' or 'javascript:void(0)' in href or 'event.preventDefault()' in onclick: # pyright: ignore[reportOperatorIssue]
            links_as_buttons.append({
                'html': str(link)[:100] + '...' if len(str(link)) > 100 else str(link),
                'onclick': onclick,
                'href': href,
                'text': link.get_text().strip()
            })
    
    return links_as_buttons

# Función para convertir enlaces a botones
def convert_links_to_buttons(html_content):
    """Convierte etiquetas <a> usadas como botones a elementos <button>"""
    soup = BeautifulSoup(html_content, 'html5lib')
    
    # Patrón para detectar enlaces que se comportan como botones
    for link in soup.find_all('a'):
        href = link.get('href', '')
        onclick = link.get('onclick', '')
        
        if href == '#' or 'javascript:void(0)' in href or 'event.preventDefault()' in onclick: # pyright: ignore[reportOperatorIssue]
            # Crear un nuevo botón
            button = soup.new_tag('button')
            button['type'] = 'button'
            
            # Copiar atributos y clases
            for attr, value in link.attrs.items():
                if attr not in ['href', 'role']:
                    button[attr] = value
            
            # Mantener las clases y añadir estilos si es necesario
            if link.get('class'):
                button['class'] = link['class']
            
            # Copiar el contenido interno
            button.extend(link.contents)
            
            # Reemplazar el enlace por el botón
            link.replace_with(button)
    
    return str(soup)

# Ejemplo de enlace usado como botón
link_as_button = """<a href="#" 
   class="btn btn-sm btn-outline-info"
   onclick="
       event.preventDefault(); 
       if (typeof showPyWebViewDocument !== 'undefined') { 
           showPyWebViewDocument('/admin/s3/document.pdf', 'document.pdf'); 
       } else { 
           showDocumentModal('/admin/s3/document.pdf', 'document.pdf'); 
       }
   ">
    <i class="fas fa-external-link-alt"></i> document.pdf...
</a>"""

# Encontrar enlaces usados como botones en el ejemplo
found_links = find_links_used_as_buttons(link_as_button)
print(f"Enlaces usados como botones: {len(found_links)}")

# Convertir enlaces a botones
converted_html = convert_links_to_buttons(link_as_button)

# Mostrar la comparación
print("\nENLACE ORIGINAL (usado como botón):")
print(link_as_button)
print("\nCONVERTIDO A BOTÓN (semánticamente correcto):")
print(converted_html)

## 7. Limpieza de Código Comentado

SonarLint detectó código HTML comentado que debería eliminarse para mejorar la mantenibilidad: